<a href="https://colab.research.google.com/github/Lhn91/ARTIFICIAL_INTELLIGENCE_IoT_Smart-Home/blob/main/SPE_buffet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Mục tiêu và xác định hệ thống
- **Mục tiêu**: đo lường [response time] toàn hệ, thông lượng, sử dụng tài nguyên, tỉ lệ bị chặn khi giới hạn tối đa 200 người trong hệ; tìm cấu hình loại bỏ nút nghẽn.
- **Boundary**: bắt đầu tại SOURCE (phát sinh khách) → Entrance1/2 → vòng quầy A=B=C (mỗi món là một lượt)
- **Discipline**: FCFS toàn hệ; phân phối đến/phục vụ với (M/M). Mạng mở theo Jackson.


## 2) Services & outcomes

- **Service**: "phục vụ khách dùng buffet qua nhiều nút xếp hàng."
- **Outcomes**:
    
    (i) dịch vụ phục vụ và rời hệ;

    (ii) tùy chọn có nút nếu hạn K hoặc toàn hệ;
                
    (iii) quay vòng ăn thêm. Các outcome này quyết định metrics và workload cần đo.


## 3) Lựa chọn metrics
- **Thời gian**: r (response time từng khách, từ vào hệ đến rời hệ), w (waiting)
- **Số lượng**: E[n], E[nq] tính từ hệ và toàn hệ.
- **Tốc độ**: throughput từng nút.
- **Tài nguyên**: $ \ U_i $ (utilization từng quầy/cổng thứ i)
- **Khả dụng**:  Xác suất bị từ chối phục vụ tại Node, xác suất bị từ chối phục vụ trong hệ thống.

## 4) Các tham số (system vs. workload)

- **System params**:
    - Entrance 1,2: M/M/c (mặc định B = ∞,  = ∞)
    - Grill,Beverage,Desert: M/M/c/B/K (mặc định cA = 3, cB = 3, cC = 3, B=10, K = 300)
    - Service rates: µA = 30/h (2'), µB = 20/h (3.33'), µC = 15/h (4')
    - Routing sau mỗi món: 1: 20% Exit; 20% → quầy (i); 30% → 2 quầy còn lại.
    - System capacity: tối đa 200 người trong hệ.

**Routing (feedback) after each dish**
- Từ các nút \({A,B,C\}\): **Exit 20%**, **quay lại nút \(i\) 20%**, **đến nút khác với tỉ lệ 30%**.
- Transition among stations (excluding Exit):
P=
\begin{bmatrix}
0.2&0.3&0.3\\
0.3&0.2&0.3\\
0.3&0.3&0.2
\end{bmatrix}

- **Workload params**:
    - SOURCE: Poisson với λ (mặc định 10 khách/giờ).
    - Split vào quầy: pE1 = pE2 = 0.5.
    - Luật đầu chọn quầy: α = (1/3, 1/3, 1/3).


## 5) Các nhân tố mô phỏng (có thể thay đổi)

- Traffic intensity với quầy tương ứng: $$ \rho_A, \rho_B, \rho_C $$
- Phục vụ: µA, µB, µC (đào tạo/quy trình).
- Số món ăn: K_i.
- Hành vi khách: α (luật đầu), matrix feedback (20/20/30/30).


## 6) Select evaluation technique

- **Discrete Event Simulation (SimPy)** là kỹ thuật chính; kiểm chứng bằng phân tích xếp hàng cơ bản (Little's law, M/M/c).


## 7) Lựa chọn workload (ít nhất 2 loại)

- **WL-1 (baseline)**: λ = 10/h.
<!-- α = 20/20/30/30. -->
- **WL-2 (cao điểm)**: λ tăng tuyến tính lên 16/h trong 90 phút;  (món A hot); còn lại giữ nguyên.
<!-- α = (0.5, 0.3, 0.2) -->
<!-- - **WL-3 (nhạy cảm rồi)**: xác suất Exit sau món = 30% (giam vòng lặp). -->


## 8) thiết kế thí nghiệm

#  Cấu trúc hàng đợi (Kendall) & nút

- **SOURCE → ENTRANCE 1,2 → 3 quầy A,B,C (feedback) → EXIT**.
- **Ký hiệu**: tất cả FCFS.
  - **Entrance j**: $$ M/M/c_{E_j}/K_{E_j} \text{mặc định } c_{E_1}=c_{E_2}=1, K=\infty, \mu_E = 30/\text{giờ} (≈ 2 phút). $$
  - **A**: $$ M/M/c_A/K_A \text{ mặc định } c_A=3, \mu_A = 20/\text{giờ} (≈ 3 phút).$$
  - **B**: $$ M/M/c_B/K_B \text{ mặc định } c_B=3, \mu_B = 18/\text{giờ} (≈ 3.333 phút). $$
  - **C**: $$ M/M/c_C/K_C \text{mặc định } c_C=3, \mu_C = 15/\text{giờ} (≈ 4 phút). $$
- **Chọn quầy ở lượt đầu**: đều nhau $$ \boldsymbol{\alpha}=(1/3,\,1/3,\,1/3). $$

# Định tuyến với “ăn tiếp” sau mỗi món

Sau khi hoàn tất một món tại quầy \(i\):
- 20% **rời hệ (EXIT)**,
- 20% **quay lại chính quầy \(i\)**,
- 30% **sang quầy còn lại 1**,
- 30% **sang quầy còn lại 2**.

**Ma trận chuyển giữa các quầy (không gồm EXIT)**:
$$
R=\begin{bmatrix}
0.2 & 0.3 & 0.3\\
0.3 & 0.2 & 0.3\\
0.3 & 0.3 & 0.2
\end{bmatrix},\qquad P(\text{EXIT}\mid i)=0.2.
$$

Mỗi hàng có tổng \(0.8\).

#  “Visit ratios” (số lượt kỳ vọng tới từng quầy)

Giả sử khách sau **ENTRANCE** vào lượt đầu chọn quầy theo $$\boldsymbol{\alpha}=[\alpha_A,\alpha_B,\alpha_C]$$ (tổng 1).

Kỳ vọng **tổng số lượt** tới từng quầy mỗi khách $$\mathbf{v_i}$$ là nghiệm của:
$$
\mathbf{v}=\boldsymbol{\alpha}+\mathbf{v}R
\quad\Longrightarrow\quad
\mathbf{v}=\boldsymbol{\alpha}(I-R)^{-1}.
$$

Đây là công thức chuẩn của **luồng bên trong** mạng Jackson (ở đây \(R\) là ma trận chuyển giữa các quầy, còn “EXIT” là hấp thụ).

**Hệ quả nhanh**: tổng số món trung bình mỗi khách ăn
$$
\mathbb{E}[N_{\text{món}}]=\mathbf{1}^\top\mathbf{v}=\frac{1}{0.2}=5,
$$
vì sau mỗi món có 20% rời hệ (biến hình học).

<!-- - **Tốc độ đến cổng**:
$$ Entrance: A1 = P_E1\lambda, A2 = P_{E2}\lambda. $$ -->

<!-- - **M/M/c**: tính

$$ E[T_i] bằng Erlang-C; tổng E[R] = E[T] + \sum_i v_i E[T_i]. Định nghĩa r, w, s & luật Little để kiểm tra. $$ -->

# Tốc độ đến hiệu dụng ở từng quầy

$$  \lambda \text{ là tốc độ khách được nhận vào hệ}. $$

Khi đó:
$$
\lambda_A=\lambda\,v_A,\qquad
\lambda_B=\lambda\,v_B,\qquad
\lambda_C=\lambda\,v_C.
$$

Tại **ENTRANCE**:
$$
\lambda_{E1}=p_{E1}\lambda,\qquad
\lambda_{E2}=p_{E2}\lambda,\qquad
(p_{E1}+p_{E2}=1).
$$

#  Thời gian tại từng nút (M/M/c – Erlang-C)

$$
\text{Với quầy } i \text{ có } c_i \text{server phục vụ, khả năng phục vụ } u_i.
$$
$$ - Đặt  \rho_i=\dfrac{\Lambda_i}{c_i\mu_i} \text{(utilization mỗi quầy, yêu cầu) } \rho_i<1. $$


$$
\text{Gọi } C(c_i,\rho_i) \text{là xác suất phải chờ của Erlang-C.}
$$
Thời gian trung bình tại quầy \(i\):
$$
\mathbb{E}[T_i]
= \underbrace{C(c_i,\rho_i)\,\frac{1}{\,c_i\mu_i-\lambda_i\,}}_{\mathbb{E}[W_{q,i}]}
+ \frac{1}{\mu_i}.
$$
$$
\text{Tương tự, với ENTRANCE j: dùng} M/M/c_{E_j} \text{với } \lambda_{E_j}, c_{E_j}, \mu_E \text{để tính } \mathbb{E}[T_{E_j}].
$$

#  Ràng buộc “tối đa 200 người” (toàn hệ)

Xem toàn hệ như “hộp đen”, áp dụng Little’s Law:
$$
\mathbb{E}[N] \;=\; \lambda_{\text{hiệu dụng}}\;\mathbb{E}[r] \;\le\; 200.
$$



#  Response time của một người & trung bình hệ

- **Cho từng khách**: khách có thể quay lại nhiều lần,
$$
r \;=\; T_{\text{Entrance}} \;+\; \sum_{\ell=1}^{N_{\text{món}}} T_{i(\ell)}^{(\ell)}.
$$
Ở đây r là **tổng response time** qua số lượt thực tế của người đó (log được trong mô phỏng: vào → ra **cuối cùng**).

- **Kỳ vọng toàn hệ** (tuyến tính theo kỳ vọng):
$$
\mathbb{E}[r]\;\approx\; \mathbb{E}[T_E]
\;+\;\sum_{i\in\{A,B,C\}} v_i\,\mathbb{E}[T_i],
\qquad
\mathbb{E}[T_E]=p_{E1}\mathbb{E}[T_{E1}] + p_{E2}\mathbb{E}[T_{E2}],
$$
trong đó $$v_i$$ là **số lượt kỳ vọng** tới quầy \(i\) của **một khách**.




**Các thư viện cần thiết cho bài tập lớn:**

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import Dict
from enum import Enum

**`Section`:** Section là một enum class sẽ là các lựa chọn của khách hàng quyết định ăn món nào hoặc rời khỏi nhà hàng



In [ ]:
class Section(Enum):
    SINK      = 1
    GRILL     = 2
    BEVERAGE  = 3
    DESSERT   = 4


In [ ]:
#parameters for Entrance
maxEntranceServer    = 1
numOfEntrance        = 2

#paremeters food queue(multiple-queueing system)
maxFoodServer    = 3
queueFoodSize    = 10
numOfDishes      = 300
inf              = np.inf

#parameters for customer's decisions:
probGrill       = 0.3
probBeverage    =  0.2
probDessert     = 0.3
probExit        = 0.2

# Grill, Beverage, Dessert
numOfFoodQueue  = 3

MMcNode:
*   MMcNode là một dataclass có arrival process và service time phân phối mũ
*   Các thuộc tính trong MMcNode là các chỉ số của Kendall Notation ngoại trừ  `lam` và `mu` sẽ lần là `arrival rate` và `service rate`




In [ ]:
@dataclass
class MMcNode:
    name: str
    lamda: float
    mu: float
    c: int
    B: int|float = inf
    K: int|float = inf
    discipline: str = "FCFS"

    def lamda_nodeith(): pass

Customer:
*   `decision`: Quyết định của khách hàng sẽ ăn món gì hay rời quán
*   `responseTime` : Thời gian của 1 khách trong hệ thống tính từ Entrance
*   `makeDecision`: Phương thức đưa ra quyết định của khách hàng dựa trên xác suất







In [ ]:
class customer:
  responseTime: float
  decision: Section

  def makeDecision(probGrill, probBeverage, probDessert, probExit): pass


Source:
*   Source sẽ là nguồn cung cấp số job cần được phục phụ (Số khách và nhà hàng)
*   Source sẽ tạo ra một mảng khách hàng bao gồm các thông tin của từng khách hàng(món muốn ăn...) để gửi vào entrance



In [ ]:
class Source:
    numOfCus: int
    def numOfCustomerIn(): pass # lambda

Entrance(M/M/1):
*   Entrance sẽ là được ánh xạ từ quầy check-in để nhận hoặc từ chối jobs
*   `numOfJobRejected`: Thuộc tính số  job không được server (vì vượt quá `system_cap`)


In [ ]:
class Entrance(MMcNode):
    def __init__(self, name: str, lam: float, mu: float, c: int=maxEntranceServer,
                B: int = inf, K: int = inf, discipline: str = "FCFS"):
        super().__init__(name, lam, mu, c, B, K, discipline)

    # parameters for distribution
    interArrivalTime: float   #1/lambda
    serviceTime: float        #1/service rate

    # Metrics in queue-level
    numOfJobRejected: int
    meanWaitingTime: float
    arrivalRate: float
    serviceRate: float
    trafficIntensity: float

FoodQueue(M/M/3/10/300):
*   FoodQueue là một Abstract Class có các thuộc tính của một quầy đồ ăn
*   3 queue `Grill`, `Beverage`, `Dessert` sẽ được kế thừa từ FoodQueue

In [ ]:
class FoodQueue(MMcNode):
    def __init__(self, name: str, lam: float, mu: float, c: int=maxFoodServer,
                B: int = queueFoodSize, K: int = numOfDishes, discipline: str = "FCFS"):
        super().__init__(name, lam, mu, c, B, K, discipline)

    # parameters for distribution
    interArrivalTime: float
    serviceTime: float

    # Metrics in queue-level:
    meanWaitingTime: float
    arrivalRate: float
    serviceRate: float
    trafficIntensity: float


# Create 3 types of queue class including Grill, Beverage, Dessert inherited from FoodQueue
class Grill(FoodQueue):    pass
class Beverage(FoodQueue): pass
class Dessert(FoodQueue):  pass



Simulation sẽ mô phỏng hệ thống hoạt động tại 2 thời điểm (bình thường, cao điểm)
So sánh các metrics giữa 2 workloads bao gồm :
*   `probOfJobRejected`: Tỷ lệ job bị từ chối
*   `probOfOutFood`: Tỷ lệ khách hàng bị hết món


*   `arrivalEntrance(Source)`: Phương thức có job đến khi arrivalTime đạt và nhận hoặc từ chối jobs từ `Source`

*   `schedule_Entrance()`: Phương thức sau khi jobs được nhận và sẽ được scheduled vào các Entrance

*   `schedule_Food(Customer)`: Phương thức schedule khách hàng vào FoodQueue theo `decision` của từng khách.
*   `beginService(self)`: Phương thức phục vụ khách dựa trên `serviceTime`


*   `endService(self)`: Phương thức kết thúc quá trình phục vụ





In [ ]:
class Simulation:
    def __init__(self, entrances: Dict[str, Entrance], food_queues: Dict[str, FoodQueue],
                          source: Source, timeout = 3600):
        self.source = source
        self.entrances = entrances
        self.food_queues = food_queues
        self.timeout = timeout

    system_cap: int
    numOfJobInSystem: int

    #Perfomance Metrics needed to be evaluated in system-level:
    probOfJobRejected: float      # Self-selected
    probOfOutFood:  float         # Self-selected
    meanResponseTime: float       # Time

    def arrivalEntrance(Source): pass

    def schedule_Entrance(): pass

    def schedule_Food(Customer): pass

    def beginService(self): pass

    def endService(self): pass

    def run(self): pass

    def report(self): pass

In [ ]:
if __name__ == "__main__": pass
